In [3]:
import tensorflow as tf
import numpy as np
# load data
#train_data=np.load("train_data.npy")
#train_label=np.load("train_label.npy")
#test_data=np.load("test_data.npy")
#test_label=np.load("test_label.npy")
train_data=np.load("c_train_data.npy")
train_label=np.load("c_train_label.npy")
test_data=np.load("c_test_data.npy")
test_label=np.load("c_test_label.npy")
print("read data ok")

/usr/local/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


read data ok


In [6]:
img_size=32
batch_size=128
img_channel=3
acc_max=0
conv_kernel_1=64
conv_kernel_2=128
conv_kernel_3=256
kernel_size_1=5
kernel_size_2=5
kernel_size_3=3
keep_prob_use=0.7
length_of_fc1=(img_size//4)*(img_size//4)*conv_kernel_3
fc1_num=1600
num_batch=50000//batch_size
x=tf.placeholder(tf.float32,[None,img_size,img_size,3])
y=tf.placeholder(tf.float32,[None,10])
keep_prob=tf.placeholder(tf.float32)
#w=tf.Variable(tf.zeros([img_size,img_size,3,100]))
#b=tf.Variable(tf.zeros([100]))

def weight_variable(shape,name_use):
    initial=tf.truncated_normal(shape,stddev=0.01)
    return tf.Variable(initial,name=name_use)
def biase_variable(shape,name_use):
    init=tf.constant(0.1,shape=shape)
    return tf.Variable(init,name=name_use)

def conv_2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def max_pool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#输入层
x_image=tf.reshape(x,[-1,img_size,img_size,img_channel])
#
w_conv1=weight_variable([kernel_size_1,kernel_size_1,img_channel,conv_kernel_1],"w_conv1")  # 32 kernels extract feature from 3 plane
b_conv1=biase_variable([conv_kernel_1],"b_conv1")
h_conv1=tf.nn.leaky_relu(conv_2d(x_image,w_conv1)+b_conv1)
h_pool1=max_pool(h_conv1)


w_conv2=weight_variable([kernel_size_2,kernel_size_2,conv_kernel_1,conv_kernel_2],"w_conv2")
b_conv2=biase_variable([conv_kernel_2],"b_conv2")
h_conv2=tf.nn.leaky_relu(conv_2d(h_pool1,w_conv2)+b_conv2)
h_pool2=max_pool(h_conv2)

w_conv3=weight_variable([kernel_size_3,kernel_size_3,conv_kernel_2,conv_kernel_3],"w_conv3")
b_conv3=biase_variable([conv_kernel_3],"b_conv3")
h_conv3=tf.nn.leaky_relu(conv_2d(h_pool2,w_conv3)+b_conv3)


w_fc1_buff=weight_variable([length_of_fc1,fc1_num],"w_fc1_buff")
b_fc1=biase_variable([fc1_num],"b_fc1")
w_fc1 = tf.nn.dropout(w_fc1_buff, keep_prob) 

h_pool2_flat=tf.reshape(h_conv3,[-1,length_of_fc1])
h_fc1=tf.nn.leaky_relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)

w_fc2_buff=weight_variable([fc1_num,10],"w_fc2_buff")
b_fc2=biase_variable([10],"b_fc2")
w_fc2 = tf.nn.dropout(w_fc2_buff, keep_prob) 
##
saver=tf.train.Saver()
prediction=tf.nn.softmax(tf.matmul(h_fc1,w_fc2)+b_fc2)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#tf.nn.sparse_softmax_cross_entropy_with_logits
train_step=tf.train.AdamOptimizer(0.9*(1e-4)).minimize(cross_entropy)
#结果存放在布尔变量中
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
##
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver=tf.train.Saver()
    for epoth in range(21):
        for batch in range(num_batch):
            if batch<(num_batch-1):
                batch_x=train_data[(batch*batch_size):((batch+1)*batch_size)]
                batch_y=train_label[(batch*batch_size):((batch+1)*batch_size)]
                sess.run(train_step,feed_dict={x:batch_x,y:batch_y,keep_prob:keep_prob_use})
        #acc=sess.run(accuracy,feed_dict={x:test_pix,y:test_label})
        acc=sess.run(accuracy,feed_dict={x:test_data,y:test_label,keep_prob:keep_prob_use})
        acc_data=sess.run(accuracy,feed_dict={x:train_data[0:10000],y:train_label[0:10000],keep_prob:keep_prob_use})
        print("iter: "+str(epoth)+"acc  "+str(acc)+"  acc_data"+str(acc_data))  
        if acc>acc_max:
            acc_max=acc
            saver.save(sess,"checkpoint_dir/cifar10_test")
print("ok")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

iter: 0acc  0.4359  acc_data0.4589
iter: 1acc  0.5112  acc_data0.5317
iter: 2acc  0.6082  acc_data0.63
iter: 3acc  0.6293  acc_data0.6433
iter: 4acc  0.6532  acc_data0.6873
iter: 5acc  0.6451  acc_data0.6818
iter: 6acc  0.6675  acc_data0.7263
iter: 7acc  0.6646  acc_data0.7245
iter: 8acc  0.6933  acc_data0.7574
iter: 9acc  0.6903  acc_data0.7635
iter: 10acc  0.7005  acc_data0.7745
iter: 11acc  0.7005  acc_data0.7875
iter: 12acc  0.7016  acc_data0.7865
iter: 13acc  0.7155  acc_data0.8133
iter: 14acc  0.7052  acc_data0.8116
iter: 15acc  0.7194  acc_data0.8164
iter: 16acc  0.7081  acc_data0.8286
iter: 17acc  0.7018  acc_data0.8147
iter: 18acc  0.7265  acc_data0.8479
iter: 19acc  0.7174  acc_data0.84
iter: 20acc  0.7195  acc_data0.8429
ok


In [ ]:
iter: 0acc  0.4292  acc_data0.4252
iter: 1acc  0.4988  acc_data0.498
iter: 2acc  0.5403  acc_data0.5569
iter: 3acc  0.6193  acc_data0.6454
iter: 4acc  0.6035  acc_data0.6253
iter: 5acc  0.6323  acc_data0.6584
iter: 6acc  0.628  acc_data0.678
iter: 7acc  0.6623  acc_data0.7295
iter: 8acc  0.687  acc_data0.7407
iter: 9acc  0.6961  acc_data0.7572
iter: 10acc  0.6909  acc_data0.7652
iter: 11acc  0.7084  acc_data0.7854
iter: 12acc  0.7025  acc_data0.7795
iter: 13acc  0.7131  acc_data0.7967
iter: 14acc  0.7128  acc_data0.7985
iter: 15acc  0.7213  acc_data0.8095
iter: 16acc  0.7198  acc_data0.8099
iter: 17acc  0.711  acc_data0.8148
iter: 18acc  0.7234  acc_data0.8208
iter: 19acc  0.7229  acc_data0.8365
iter: 20acc  0.7294  acc_data0.8421


In [19]:
with tf.Session() as sess:
    saver=tf.train.import_meta_graph("./checkpoint_dir/cifar10.meta")
    saver.restore(sess,"./checkpoint_dir/cifar10")
    #graph=tf.get_default_graph()
    acc1=acc=sess.run(accuracy,feed_dict={x:test_data,y:test_label,keep_prob:keep_prob_use})
    print(str(acc1))

INFO:tensorflow:Restoring parameters from ./checkpoint_dir/cifar10
0.72


In [18]:
acc

0.7275